# 1) Requirements

Installs and loads all necessary python packages and libraries as detailed in 'requirements.txt':

In [ ]:
!pip install yahoo_fin
!pip install twython 

In [ ]:
import os
import requests

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from yahoo_fin import stock_info as si 
from pandas_datareader import DataReader
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

# 2) Retrieving Yahoo! Finance Buy/Sell/Hold Recommendations

Retrieves Yahoo! Finance recommendations given appropriate stock tickers. The default is securities in the S&P500 index; however, the tickers will be determined by a student's program/major in production. Results are stored in a Dataframe sorted by their recommendation (1:Strong Buy -> 5:Strong Sell):

In [108]:
# load tickers
tickers = si.tickers_sp500()
yahoo_recommendations = []

# call request from yahoo
for each_ticker in tickers:
    print("Now Querying Recommendation for: " + each_ticker)
    query_a = 'https://query2.finance.yahoo.com/v10/finance/quoteSummary/'
    query_b = '?formatted=true&crumb=swg7qs5y9UP&lang=en-US&region=US&' \
              'modules=upgradeDowngradeHistory,recommendationTrend,' \
              'financialData,earningsHistory,earningsTrend,industryTrend&' \
              'corsDomain=finance.yahoo.com'
              
    request_url =  query_a + each_ticker + query_b
    response = requests.get(request_url)

    # if response failed
    if not response.ok:
        recommendation = 0

    # response success
    try:
        result = response.json()['quoteSummary']['result'][0]
        recommendation = result['financialData']['recommendationMean']['fmt']
    except:
        recommendation = 0
    
    yahoo_recommendations.append(recommendation)

# converting to a pandas dataframe
data = {'Ticker':tickers,'Recommendations':yahoo_recommendations}
df = pd.DataFrame(data)
df['Recommendations'] = pd.to_numeric(df['Recommendations'])

# sort and filter dataframe
df = df[df.Recommendations != 0]
df.sort_values(by=['Recommendations'], ascending = True)

Now Querying: A
Now Querying: AAL
Now Querying: AAP
Now Querying: AAPL
Now Querying: ABBV
Now Querying: ABC
Now Querying: ABMD
Now Querying: ABT
Now Querying: ACN
Now Querying: ADBE
Now Querying: ADI
Now Querying: ADM
Now Querying: ADP
Now Querying: ADSK
Now Querying: AEE
Now Querying: AEP
Now Querying: AES
Now Querying: AFL
Now Querying: AIG
Now Querying: AIZ
Now Querying: AJG
Now Querying: AKAM
Now Querying: ALB
Now Querying: ALGN
Now Querying: ALK
Now Querying: ALL
Now Querying: ALLE
Now Querying: ALXN
Now Querying: AMAT
Now Querying: AMCR
Now Querying: AMD
Now Querying: AME
Now Querying: AMGN
Now Querying: AMP
Now Querying: AMT
Now Querying: AMZN
Now Querying: ANET
Now Querying: ANSS
Now Querying: ANTM
Now Querying: AON
Now Querying: AOS
Now Querying: APA
Now Querying: APD
Now Querying: APH
Now Querying: APTV
Now Querying: ARE
Now Querying: ATO
Now Querying: ATVI
Now Querying: AVB
Now Querying: AVGO
Now Querying: AVY
Now Querying: AWK
Now Querying: AXP
Now Querying: AZO
Now Queryin

,Ticker,Recommendations
201,GOOG,1.5
62,BIO,1.5
271,LDOS,1.6
277,LKQ,1.6
265,KMX,1.6
...,...,...
284,LUMN,3.4
151,ED,3.5
1,AAL,3.6
498,XRX,3.9


# 3) Filter by Recommendation

Filter tickers according to the strength of its Yahoo! recommendation. In production, this will depend on a student's familiarity with trading/investing, such as if the student has experiences with shorting and holding existing positions:

In [156]:
# filter based on recommendation value
df_buy = df[df.Recommendations <= 1.5]
df_sell = df[df.Recommendations >= 4.5]
df_hold = df[df.Recommendations == 3]

# combine dataframes (for experienced investor who understands buys/sells/holds)
df_final = pd.concat([hold_df, buy_df, sell_df])
df_final.reset_index(level=0, inplace=True)

# 4) Scrub News Headlines from FinViz

Using FinViz, scrub for news headlines that relate to ticker. Results are concatenated and transformed into a DataFrame for VADER processing:

In [169]:
news = {}
parsed_news = []
columns = ['ticker', 'date', 'time', 'headlines']

finviz = 'https://finviz.com/quote.ashx?t='

# scrubs news from FinViz's site
for each_ticker in df_final['Ticker']:

    print("Now Scraping News for: " + each_ticker)
    
    url = finviz + each_ticker
    header = {'user-agent': 'my-app/0.0.1'}

    request = Request(url=url, headers=header) 
    response = urlopen(request) 

    # html resposne returning news-table
    html = BeautifulSoup(response)
    result = html.find(id='news-table')

    news[each_ticker] = result

# parses news table for headlines
for name, table in news.items():
    # values return in <tr>
    for val in table.findAll('tr'):
    
        text = val.a.get_text() 
        date_time = val.td.text.split()

        if len(date_time) == 1:
            time = date_time[0]
        else:
            date = date_time[0]
            time = date_time[1]

        each_ticker = name.split('_')[0]
        
        parsed_news.append([each_ticker, date, time, text])
        
# convert and concatenate to dataframe
scrubbed_news = pd.DataFrame(parsed_news, columns=columns)
scrubbed_news['date'] = pd.to_datetime(scrubbed_news.date).dt.date
scrubbed_news = scrubbed_news.groupby(['ticker'], as_index = False).agg({'headlines': ''.join})
scrubbed_news.insert(2, "Recommendation", df_final.Recommendations, True) 
scrubbed_news['Recommendation'] = pd.to_numeric(scrubbed_news['Recommendation'])

Now Scraping News for: AFL
Now Scraping News for: CCL
Now Scraping News for: CHRW
Now Scraping News for: CINF
Now Scraping News for: CMA
Now Scraping News for: DISCK
Now Scraping News for: FLIR
Now Scraping News for: FRT
Now Scraping News for: L
Now Scraping News for: MMM
Now Scraping News for: NTRS
Now Scraping News for: OXY
Now Scraping News for: VTR
Now Scraping News for: WBA
Now Scraping News for: XYL
Now Scraping News for: BIO
Now Scraping News for: GOOG


# 5) VADER Model Sentiment Analysis

Conduct VADER sentiment analysis on scrubbed news headlines and compute a polarity compound score:

In [185]:
# compute VADER compound score
scores = scrubbed_news['headlines'].apply(SentimentIntensityAnalyzer().polarity_scores).tolist()

# combine into one dataframe and sort for results
df_with_scores = pd.DataFrame(scores)
df_results = scrubbed_news.join(df_with_scores, rsuffix='_right')
df_results.sort_values(by='compound', ascending=False, inplace=True)
df_results.reset_index(level=0, inplace=True)

# 6) Print Results

In [188]:
for i in range(0, len(df_results)):
    if df_results["Recommendation"][i] <= 1.5:
        print("Buy: " + df_results['ticker'][i])
    elif df_results["Recommendation"][i] >= 4.5:
        print("Sell: " + df_results['ticker'][i])
    elif df_results["Recommendation"][i] == 3:
        print("Hold: " + df_results['ticker'][i])

Hold: NTRS
Hold: FRT
Hold: FLIR
Hold: MMM
Hold: AFL
Hold: CCL
Hold: OXY
Hold: BIO
Hold: CMA
Buy: XYL
Hold: VTR
Buy: WBA
Hold: DISCK
Hold: CHRW
Hold: L
Hold: CINF
Hold: GOOG
